In [2]:
import dataset
import json
import csv
import tweepy
from sqlalchemy.exc import ProgrammingError
import pandas as pd
import dataset

In [1]:
db = dataset.connect("DB_URL")

NameError: name 'dataset' is not defined

In [6]:
#insert your credentials here
import os
from dotenv import load_dotenv

load_dotenv()


CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_KEY = os.getenv('ACCESS_KEY')
ACCESS_SECRET = os.getenv('ACCESS_SECRET')

In [7]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        filter_words = ["police", "officer", "cop"]
        conditions = (not 'RT @' in status.text) and any(word in status.text for word in filter_words)
        if conditions:
            description = status.user.description
            loc = status.user.location
            text = status.text
            coords = status.coordinates
            geo = status.geo
            name = status.user.screen_name
            user_created = status.user.created_at
            id_str = status.id_str
            created = status.created_at
            source = status.user.url
            language = status.lang

            if geo is not None:
                geo = json.dumps(geo)

            if coords is not None:
                coords = json.dumps(coords)

            table = db["tweets"]
            try:
                table.insert(dict(
                    user_description=description,
                    user_location=loc,
                    coordinates=coords,
                    text=text,
                    geo=geo,
                    user_name=name,
                    user_created=user_created,
                    id_str=id_str,
                    created=created,
                    source = source,
                    language = language,
                    ))
            except ProgrammingError as err:
                print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #return False in on_data disconnects the stream
            return False

In [10]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

In [8]:
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=["police", "cop", "officer"])

KeyboardInterrupt: 

In [17]:
df_raw = pd.DataFrame(db['tweets'])
df_raw = df_raw[df_raw['language'].isin(["en", "und"]) ]

In [18]:
df = df_raw[["id_str", "text"]]
df['reddit'] = 0
df.rename(columns={'id_str': 'ids'}, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
df

,ids,text,reddit
0,1363985604313235459,@boblister_poole There are far too many senior...,0
2,1363985607186288647,"@curlycomedy Wait, can we at least talk about ...",0
3,1363985611795873796,#BlackLivesMatter,0
4,1363985612768862210,@TheMQCWestside @jemelehill Crazy I don't trus...,0
5,1363985619060363267,NIKKI IS ABSOLUTELY CORRECT! AMERICA HAD TO BE...,0
...,...,...,...
6161,1364030259171971076,@narsheviking @heghapoghagan @Raz_Libar @n0cla...,0
6162,1364030261483208710,@Lauren_Southern This is a reminder that the p...,0
6163,1364030261906857984,@nytimes Pelosi set the attack up days earlier...,0
6164,1364030277463519234,So i started playing and then some time in Dec...,0


In [20]:
df_reddit = pd.read_csv('reddit_tweets.csv')

In [21]:
df_combined = pd.concat([df_reddit, df])

In [22]:
df_combined

,ids,text,reddit
0,1266136557871869952,Police in NYC made several arrests during a pr...,1
1,1266159669262893057,Calls for justice for George Floyd. Protesters...,1
2,1266555286678048770,NYPD just casually slamming a dude with a car ...,1
3,1266540710188195843,Update: Got her permission with a fuck yeah. T...,1
4,1266529475757510656,NYPD officer just called a female protester a ...,1
...,...,...,...
6161,1364030259171971076,@narsheviking @heghapoghagan @Raz_Libar @n0cla...,0
6162,1364030261483208710,@Lauren_Southern This is a reminder that the p...,0
6163,1364030261906857984,@nytimes Pelosi set the attack up days earlier...,0
6164,1364030277463519234,So i started playing and then some time in Dec...,0


In [23]:
#created CSV

df_combined.to_csv("combined_tweets.csv", index=False)